In [1]:
import re
from itertools import combinations
from collections import defaultdict

def parse_input(file_path):
    wire_values = {}
    gate_operations = []
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    
    for line in lines:
        if not line.strip():
            continue  # Skip empty lines
        if ':' in line:
            # Initial wire value
            wire, value = line.split(':')
            wire = wire.strip()
            value = int(value.strip())
            wire_values[wire] = value
        elif '->' in line:
            # Gate definition
            gate_operations.append(line.strip())
        else:
            continue  # Skip any unexpected lines
    
    return wire_values, gate_operations

def evaluate_gates(wire_values, gate_operations):
    gate_dict = {}
    
    # Parse gates and build a dictionary mapping output wire to its operation and inputs
    for gate in gate_operations:
        match = re.match(r'(\w+) (AND|OR|XOR) (\w+) -> (\w+)', gate)
        if match:
            in1, op, in2, out = match.groups()
            gate_dict[out] = (in1, op, in2)
        else:
            raise ValueError(f"Invalid gate format: {gate}")
    
    # Iteratively compute wire values
    pending = set(gate_dict.keys())
    while pending:
        progress = False
        to_remove = set()
        for out in pending:
            in1, op, in2 = gate_dict[out]
            if in1 in wire_values and in2 in wire_values:
                val1 = wire_values[in1]
                val2 = wire_values[in2]
                if op == 'AND':
                    result = val1 & val2
                elif op == 'OR':
                    result = val1 | val2
                elif op == 'XOR':
                    result = val1 ^ val2
                else:
                    raise ValueError(f"Unknown operation: {op}")
                wire_values[out] = result
                to_remove.add(out)
                progress = True
        if not progress:
            raise ValueError("Cannot resolve all gates. Possible circular dependency or missing inputs.")
        pending -= to_remove
    return wire_values

def get_z_wires(wire_values):
    # Extract wires starting with 'z'
    z_wires = {wire: val for wire, val in wire_values.items() if wire.startswith('z')}
    return z_wires

def get_expected_sum(wire_values, z_wires):
    # Extract x and y wires
    x_wires = {wire: val for wire, val in wire_values.items() if wire.startswith('x')}
    y_wires = {wire: val for wire, val in wire_values.items() if wire.startswith('y')}
    
    # Determine the maximum bit index
    max_bit_x = max(int(wire[1:]) for wire in x_wires.keys()) if x_wires else -1
    max_bit_y = max(int(wire[1:]) for wire in y_wires.keys()) if y_wires else -1
    max_bit = max(max_bit_x, max_bit_y)
    
    # Build binary numbers
    x_bits = [x_wires.get(f'x{str(i).zfill(2)}', 0) for i in range(max_bit + 1)]
    y_bits = [y_wires.get(f'y{str(i).zfill(2)}', 0) for i in range(max_bit + 1)]
    
    # Convert to integers
    x_num = sum(bit << i for i, bit in enumerate(x_bits))
    y_num = sum(bit << i for i, bit in enumerate(y_bits))
    
    # Compute sum
    sum_num = x_num + y_num
    
    # Determine the number of z wires
    num_z_wires = len(z_wires)
    
    # Convert sum to binary bits
    sum_bits = []
    temp = sum_num
    for _ in range(num_z_wires):
        sum_bits.append(temp & 1)
        temp >>= 1
    # Handle carry-over bits beyond current z wires by padding with zeros if necessary
    while len(sum_bits) < num_z_wires:
        sum_bits.append(0)
    
    # Map sum bits to z wires
    expected_z_wires = {}
    for i, bit in enumerate(sum_bits):
        wire = f'z{str(i).zfill(2)}'
        expected_z_wires[wire] = bit
    return expected_z_wires

def find_discrepancies(z_wires, expected_z_wires):
    discrepancies = {}
    for wire, expected_val in expected_z_wires.items():
        actual_val = z_wires.get(wire, 0)
        if actual_val != expected_val:
            discrepancies[wire] = (actual_val, expected_val)
    return discrepancies

def identify_swapped_wires(discrepancies, expected_z_wires):
    # List of incorrect z wires
    incorrect_z_wires = list(discrepancies.keys())
    
    if len(incorrect_z_wires) != 8:
        print(f"Expected 8 incorrect z wires, found {len(incorrect_z_wires)}.")
        # Proceeding under the assumption that the discrepancy count is correct
        # If not, additional investigation is needed
        # For now, we proceed only if there are exactly 8 discrepancies
        if len(incorrect_z_wires) < 8:
            raise ValueError("Less than 8 discrepancies found. Unable to identify all swapped wires.")
        elif len(incorrect_z_wires) > 8:
            print("More than 8 discrepancies found. Additional logic may be required.")
            # Optionally, proceed with the first 8
            incorrect_z_wires = incorrect_z_wires[:8]
    
    # Now, find four pairs among these eight wires that, when swapped, fix the discrepancies.
    # We'll iterate through all possible unique pairings of the eight wires into four pairs.
    
    # Generate all possible unique pairings
    def all_unique_pairings(wires):
        if not wires:
            yield []
            return
        first = wires[0]
        for i in range(1, len(wires)):
            pair = (first, wires[i])
            for rest in all_unique_pairings(wires[1:i] + wires[i+1:]):
                yield [pair] + rest
    
    # Iterate through all possible unique pairings
    for pairing in all_unique_pairings(incorrect_z_wires):
        if len(pairing) != 4:
            continue
        # For each pairing, check if swapping their actual values matches the expected values
        swap_correct = True
        for wire1, wire2 in pairing:
            actual1, expected1 = discrepancies[wire1]
            actual2, expected2 = discrepancies[wire2]
            if actual1 != expected2 or actual2 != expected1:
                swap_correct = False
                break
        if swap_correct:
            # Found the correct pairing
            swapped_wires = []
            for pair in pairing:
                swapped_wires.extend(pair)
            # Sort the wire names
            swapped_wires_sorted = sorted(swapped_wires)
            # Join with commas
            result = ','.join(swapped_wires_sorted)
            return result
    
    # If no valid pairing found
    return "No valid swapped pairs found."

def main():
    input_file = 'input.txt'
    wire_values, gate_operations = parse_input(input_file)
    
    # Simulate the gates
    wire_values = evaluate_gates(wire_values, gate_operations)
    
    # Extract z wires
    z_wires = get_z_wires(wire_values)
    
    # Compute expected sum
    expected_z_wires = get_expected_sum(wire_values, z_wires)
    
    # Identify discrepancies
    discrepancies = find_discrepancies(z_wires, expected_z_wires)
    
    print(f"Number of discrepancies found: {len(discrepancies)}")
    if len(discrepancies) != 8:
        print("Unexpected number of discrepancies. The system expects exactly 8 discrepancies due to four swapped pairs.")
        print("Proceeding with identification may not yield accurate results.")
    
    # Identify swapped wires
    try:
        swapped_wires = identify_swapped_wires(discrepancies, expected_z_wires)
        print(f"Swapped Wires: {swapped_wires}")
    except ValueError as e:
        print(f"Error: {e}")
        swapped_wires = "Error in identifying swapped wires."
    
    # Output the result
    print(swapped_wires)

if __name__ == "__main__":
    main()


Number of discrepancies found: 12
Unexpected number of discrepancies. The system expects exactly 8 discrepancies due to four swapped pairs.
Proceeding with identification may not yield accurate results.
Expected 8 incorrect z wires, found 12.
More than 8 discrepancies found. Additional logic may be required.
Swapped Wires: No valid swapped pairs found.
No valid swapped pairs found.
